In [67]:
import datetime
from pathlib import Path
from typing import Dict, List, Union

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import matplotlib.ticker as ticker
from matplotlib.patches import Patch
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm

%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams.update({'figure.max_open_warning': 0})
PARAMS = {'legend.fontsize': 'xx-large',
          'legend.title_fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(PARAMS)

TIME_FORMAT = '%Y-%m-%d'
INT_TYPE = 'int32'

In [68]:
def weighted_avg(x, weight, factor):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        tmp = x[[weight, factor]].dropna()
        weighted_sum = (tmp[weight] * tmp[factor]).sum()
        count_sum = tmp[weight].sum()
        return weighted_sum / count_sum

def load_data(locus_weights_path: Union[str, Path],
              temperature_path: Union[str, Path],
              locus_group_matching_path: Union[str, Path],
              fresh_water_dates_path: Union[str, Path],
              sw_fw_matching_path: Union[str, Path],
              sw_fw_matching_path_with_cnt: Union[str, Path],
              final_locus_weighted_path: Union[str, Path],
              mortality_path: Union[str, Path]) -> Dict[str, pd.DataFrame]:
    
    locus_weights = pd.read_csv(locus_weights_path)
    locus_weights.starttime = pd.to_datetime(locus_weights.starttime, format=TIME_FORMAT)
    locus_weights.endtime = pd.to_datetime(locus_weights.endtime, format=TIME_FORMAT)
    
    temperature = pd.read_csv(temperature_path)
#     temperature.event_date = pd.to_datetime(temperature.event_date, format=TIME_FORMAT)
#     temperature.locus_group_id=temperature.locus_group_id.astype(INT_TYPE)
#     temperature['event_year'] = temperature['event_date'].dt.year

    locus_group_matching = pd.read_csv(locus_group_matching_path)
    locus_group_matching = locus_group_matching.astype(INT_TYPE)

    fresh_water_dates = pd.read_csv(fresh_water_dates_path)
    for d in ['first_movement_date', 'first_feeding_date', 'shipout_date']:
        fresh_water_dates[d] = pd.to_datetime(fresh_water_dates[d], format=TIME_FORMAT)

    sw_fw_matching = pd.read_csv(sw_fw_matching_path)
    sw_fw_matching_with_cnt = pd.read_csv(sw_fw_matching_path_with_cnt)

    sw_fw_cols = ['target_seawater_locus_id',
                  'transport_date',
                  'ponding_date',
                  'pretransfer_fw_locus_population_id',
                  'fish_count_shipped_out',
                  'avg_weight_g_stocked']
    sw_fw_matching = sw_fw_matching[sw_fw_matching.origin_site_type == 'Freshwater'][sw_fw_cols]
    sw_fw_matching.pretransfer_fw_locus_population_id = sw_fw_matching.\
    pretransfer_fw_locus_population_id.astype(INT_TYPE)

    final_locus_weighted = pd.read_csv(final_locus_weighted_path)
    final_locus_weighted.event_date = pd.to_datetime(final_locus_weighted.event_date)
    #final_locus_weighted = None

    mortality = pd.read_csv(mortality_path)
    mortality['transfer_date'] = pd.to_datetime(mortality['transfer_date'], format=TIME_FORMAT)
    mortality['transport_year']= mortality['transfer_date'].dt.year


    dataframes = {
        'temperature': temperature,
        'locus_weigts': locus_weights,
        'locus_group_matching': locus_group_matching,
        'final_locus_weighted': final_locus_weighted,
        'fresh_water_dates': fresh_water_dates,
        'sw_fw_matching': sw_fw_matching,
        'sw_fw_matching_with_cnt': sw_fw_matching_with_cnt,
        'mortality': mortality
    }
    
    return dataframes

In [69]:
dataframes = load_data(locus_weights_path='../data/evt_movement_ratio_with_dates.csv',
                       temperature_path='../data/FW_temperature_cleared.csv',
                       locus_group_matching_path='../data/locus_locus_group_matching.csv',
                       fresh_water_dates_path='../data/FW_cycle_dates.csv',
                       sw_fw_matching_path='../data/seawater_freshwater_matching.csv',
                       sw_fw_matching_path_with_cnt='../data/sw_locus_fw_locus_population_with_counts.csv',
                       final_locus_weighted_path='../data/lw_alldates_final.csv',
                       mortality_path='../data/smolt_dataset_transfers.csv') #_until2023Feb28_narrow

temperature = dataframes['temperature']
temperature.event_date = pd.to_datetime(temperature.event_date, format=TIME_FORMAT)
temperature['event_year'] = temperature['event_date'].dt.year

locus_weights = dataframes['locus_weigts']
locus_group_matching = dataframes['locus_group_matching']
final_locus_weighted = dataframes['final_locus_weighted']
fresh_water_dates = dataframes['fresh_water_dates']
sw_fw_matching = dataframes['sw_fw_matching']
mortality = dataframes['mortality']

sw_fw_matching_with_cnt = dataframes['sw_fw_matching_with_cnt']
sw_fw_matching.transport_date = pd.to_datetime(sw_fw_matching.transport_date, format=TIME_FORMAT)
sw_fw_matching_with_cnt.transfer_date = pd.to_datetime(sw_fw_matching_with_cnt['transfer_date'], format=TIME_FORMAT)
sw_fw_matching_with_cnt['transfer_year'] = sw_fw_matching_with_cnt['transfer_date'].dt.year

In [70]:
lw_dates = locus_weights.groupby('final_locus_population_id').agg({'starttime': 'min', 'endtime': 'max'})
lw_dates.starttime = pd.to_datetime(lw_dates.starttime, format=TIME_FORMAT)
lw_dates.endtime = pd.to_datetime(lw_dates.endtime, format=TIME_FORMAT)
lw_dates['FW_cycle_length'] = (lw_dates.endtime - lw_dates.starttime).dt.days + 1  # to be checked (?)
lw_dates['starttime_year'] = lw_dates['starttime'].dt.year
lw_dates = lw_dates[lw_dates.starttime_year>=2017]  # issues with temperature readings for 2015-2016

In [6]:
# final_locus_locus_group_weighted = final_locus_weighted.merge(locus_group_matching,
#                                                               left_on='historic_locus_id',
#                                                               right_on='locus_id',
#                                                               how='left')
# final_locus_locus_group_weighted.drop(columns='locus_id',inplace=True)

### Mortality

In [44]:
fw_mortality = pd.read_csv('../data/fw_mortality.csv')
mortality_ref = pd.read_csv('../data/dict_mortality.csv', encoding="ISO-8859-1")
inventory = pd.read_csv('../data/inventory_Petrohue_UPS.csv')

inventory = inventory.groupby(['event_date', 'locus_id']).max().reset_index()

In [45]:
if False:
    fw_mortality = fw_mortality.merge(mortality_ref,
                                      left_on='mortality_reason_id',
                                      right_on='id',
                                      how='left')

In [46]:
display(fw_mortality.head())
display('Shape:', fw_mortality.shape)

,site_id,locus_id,locus_population_id,mortality_reason_id,event_date,mortality_count,mortality_weight,id,mortality_reason,mortality_reason_code,mrts_mortality_reason_id,open_count,open_weight,close_count,close_weight,degree_days,mortality_rate
0,63827,3047045,194524401,13,2023-01-04,6,3.58,13,Desadaptado,DESADA,130,45901.0,3.584349,45895.0,3.755513,14.2,0.000131
1,63827,3046733,194524700,13,2023-01-04,42,3.30,13,Desadaptado,DESADA,130,45701.0,3.297303,45659.0,3.436012,14.2,0.000919
2,63827,3050450,194524394,54,2023-01-04,19,3.44,54,Sin Causa Aparente,S/CAUS,107,46593.0,3.437077,46567.0,3.539677,14.2,0.000408
3,63827,3050450,194524394,13,2023-01-04,6,3.44,13,Desadaptado,DESADA,130,46593.0,3.437077,46567.0,3.539677,14.2,0.000129
4,63827,3050450,194524394,38,2023-01-04,1,3.44,38,Micosis,MICOSI,123,46593.0,3.437077,46567.0,3.539677,14.2,0.000021


'Shape:'

(785677, 17)

In [47]:
reason_exclude = 'Eliminación Productiva'

In [48]:
fw_mortality.columns

Index(['site_id', 'locus_id', 'locus_population_id', 'mortality_reason_id',
       'event_date', 'mortality_count', 'mortality_weight', 'id',
       'mortality_reason', 'mortality_reason_code', 'mrts_mortality_reason_id',
       'open_count', 'open_weight', 'close_count', 'close_weight',
       'degree_days', 'mortality_rate'],
      dtype='object')

In [49]:
fw_mortality = fw_mortality[fw_mortality['mortality_reason'] != reason_exclude]

In [50]:
fw_mortality.head()
display('Shape:', fw_mortality.shape)

'Shape:'

(785677, 17)

In [43]:
inventory.columns

Index(['event_date', 'locus_id', 'open_count', 'open_weight', 'close_count',
       'close_weight', 'degree_days'],
      dtype='object')

In [40]:
fw_mortality = fw_mortality.merge(inventory,
                                  on=['event_date', 'locus_id'],
                                  how='left')
fw_mortality['event_date'] = pd.to_datetime(fw_mortality['event_date'])

In [41]:
fw_mortality.head()
display('Shape:', fw_mortality.shape)

'Shape:'

(785677, 22)

In [42]:
fw_mortality.columns

Index(['site_id', 'locus_id', 'locus_population_id', 'mortality_reason_id',
       'event_date', 'mortality_count', 'mortality_weight', 'id',
       'mortality_reason', 'mortality_reason_code', 'mrts_mortality_reason_id',
       'open_count_x', 'open_weight_x', 'close_count_x', 'close_weight_x',
       'degree_days_x', 'mortality_rate', 'open_count_y', 'open_weight_y',
       'close_count_y', 'close_weight_y', 'degree_days_y'],
      dtype='object')

In [31]:
display(f'Mortality nan counts: {fw_mortality.mortality_count.isna().sum()}')
display(f'Open count nan counts: {fw_mortality.open_count.isna().sum()}')

'Mortality nan counts: 0'

AttributeError: 'DataFrame' object has no attribute 'open_count'

In [ ]:
fw_mortality['open_count'] = fw_mortality.groupby('locus_id')['open_count']\
.apply(lambda group: group.interpolate(method='linear')).reset_index(level=0, drop=True)

In [32]:
fw_mortality.columns

Index(['site_id', 'locus_id', 'locus_population_id', 'mortality_reason_id',
       'event_date', 'mortality_count', 'mortality_weight', 'id',
       'mortality_reason', 'mortality_reason_code', 'mrts_mortality_reason_id',
       'open_count_x', 'open_weight_x', 'close_count_x', 'close_weight_x',
       'degree_days_x', 'mortality_rate', 'open_count_y', 'open_weight_y',
       'close_count_y', 'close_weight_y', 'degree_days_y'],
      dtype='object')

In [20]:
display(f'Open count nan counts after interpolation: {fw_mortality.open_count.isna().sum()}')

'Open count nan counts after interpolation: 0'

In [21]:
display(fw_mortality[fw_mortality['open_count'] == 0].tail())
display(f"Open count 0 counts: {fw_mortality[fw_mortality['open_count'] == 0].shape[0]}")

,site_id,locus_id,locus_population_id,mortality_reason_id,event_date,mortality_count,mortality_weight,id,mortality_reason,mortality_reason_code,mrts_mortality_reason_id,open_count,open_weight,close_count,close_weight,degree_days
785055,63827,3047433,194473886,13,2021-08-10,2,3.30,13,Desadaptado,DESADA,130,0.0,0.0,49597.0,3.396845,12.7
785088,63827,3047490,194527095,54,2022-12-02,2,23.97,54,Sin Causa Aparente,S/CAUS,107,0.0,0.0,80588.0,24.468017,14.5
785252,63827,3047490,194528963,23,2022-12-22,263,35.40,23,Eliminación Opérculo,ElxOpe,175,0.0,0.0,42782.0,35.869279,14.5
785399,63827,3047490,194528963,22,2022-12-22,414,35.40,22,Eliminación Maduro,ElxMad,170,0.0,0.0,42782.0,35.869279,14.5
785420,63827,3047490,194528963,18,2022-12-22,420,35.40,18,Eliminación Deforme,ElxDef,176,0.0,0.0,42782.0,35.869279,14.5


'Open count 0 counts: 6621'

In [22]:
display(f"Samples with 0 open and close count: {fw_mortality[fw_mortality['open_count'] == 0][fw_mortality['close_count'] == 0].shape[0]}")
fw_mortality.drop(
    fw_mortality[fw_mortality['open_count'] == 0][fw_mortality['close_count'] == 0].index, inplace=True
)

C:\Users\dmitrii\AppData\Local\Temp\ipykernel_10352\437251776.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  display(f"Samples with 0 open and close count: {fw_mortality[fw_mortality['open_count'] == 0][fw_mortality['close_count'] == 0].shape[0]}")


'Samples with 0 open and close count: 4'

C:\Users\dmitrii\AppData\Local\Temp\ipykernel_10352\437251776.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fw_mortality[fw_mortality['open_count'] == 0][fw_mortality['close_count'] == 0].index, inplace=True


In [67]:
fw_mortality['open_count'][fw_mortality['open_count'] == 0] = fw_mortality['close_count']  # + fw_mortality['mortality_coun']
display(f"Open count 0 counts after assigning close count values: {fw_mortality[fw_mortality['open_count'] == 0].shape[0]}")

'Open count 0 counts after assigning close count values: 4'

In [68]:
fw_mortality.head()

,site_id,locus_id,locus_population_id,mortality_reason_id,event_date,mortality_count,mortality_weight,id,mortality_reason,mortality_reason_code,mrts_mortality_reason_id,open_count,open_weight,close_count,close_weight,degree_days
0,63827,3047045,194524401,13,2023-01-04,6,3.58,13,Desadaptado,DESADA,130,45901.0,3.584349,45895.0,3.755513,14.2
1,63827,3046733,194524700,13,2023-01-04,42,3.30,13,Desadaptado,DESADA,130,45701.0,3.297303,45659.0,3.436012,14.2
2,63827,3050450,194524394,54,2023-01-04,19,3.44,54,Sin Causa Aparente,S/CAUS,107,46593.0,3.437077,46567.0,3.539677,14.2
3,63827,3050450,194524394,13,2023-01-04,6,3.44,13,Desadaptado,DESADA,130,46593.0,3.437077,46567.0,3.539677,14.2
4,63827,3050450,194524394,38,2023-01-04,1,3.44,38,Micosis,MICOSI,123,46593.0,3.437077,46567.0,3.539677,14.2


In [69]:
fw_mortality[fw_mortality['locus_id'] == 3046035][fw_mortality['event_date'] == '2018-09-08']

C:\Users\dmitrii\AppData\Local\Temp\ipykernel_10352\420157352.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fw_mortality[fw_mortality['locus_id'] == 3046035][fw_mortality['event_date'] == '2018-09-08']


,site_id,locus_id,locus_population_id,mortality_reason_id,event_date,mortality_count,mortality_weight,id,mortality_reason,mortality_reason_code,mrts_mortality_reason_id,open_count,open_weight,close_count,close_weight,degree_days
376021,63858,3046035,36787705,44,2018-09-08,24,65.0,44,Otros,Otros,164,119438.0,65.000001,119395.0,65.000001,11.9
377779,63858,3046035,36787705,11,2018-09-08,19,65.0,11,Daño Mecánico,Daño M,157,119438.0,65.000001,119395.0,65.000001,11.9


In [70]:
fw_mortality['mortality_rate'] = fw_mortality['mortality_count'] / fw_mortality['open_count']
display(f"Mortality rate misasing values after dividing by 0: {fw_mortality['mortality_rate'].isna().sum()}")
display(f"Mortality rate inf values after dividing by 0: {fw_mortality[fw_mortality['mortality_rate'].isin([np.inf, -np.inf])].shape[0]}")

'Mortality rate misasing values after dividing by 0: 0'

'Mortality rate inf values after dividing by 0: 4'

In [71]:
#if False:
fw_mortality.to_csv('..//data//fw_mortality.csv', index=False)


##  OVERWRITE

In [72]:
fw_mortality = pd.read_csv('..//data//fw_mortality.csv')
fw_mortality['event_date'] = pd.to_datetime(fw_mortality['event_date'])

### By reasons

In [73]:
mortality_reasons = ['Desadaptado',
]

if True:
    fw_mortality_by_reason = {}
    for reason in mortality_reasons:
        fw_mortality_by_reason[reason] = fw_mortality[fw_mortality['mortality_reason'] == reason]
        display(f'Shape is {fw_mortality_by_reason[reason].shape} for reason {reason}')
        
# fw_mortality = fw_mortality[fw_mortality['mortality_reason'].isin(mortality_reasons)]

'Shape is (243672, 17) for reason Desadaptado'

In [74]:
if True:
    for reason in tqdm(fw_mortality_by_reason):
        fw_mortality_by_reason[reason] = fw_mortality_by_reason[reason].merge(
            final_locus_weighted,
            how='inner', 
            left_on=['event_date', 'locus_id'],
            right_on=['event_date', 'historic_locus_id']
        )

        fw_mortality_by_reason[reason]['event_year'] = fw_mortality_by_reason[reason]['event_date'].dt.year
        fw_mortality_by_reason[reason]['weighted_mortality_rate'] = fw_mortality_by_reason[reason]['weight'] * fw_mortality_by_reason[reason]['mortality_rate']

        fw_mortality_by_reason[reason] = fw_mortality_by_reason[reason].\
        groupby(['final_locus_population_id','event_date'])[['weighted_mortality_rate']]\
        .agg(lambda x: x.sum(skipna=False)).reset_index()

        fw_mortality_by_reason[reason].rename(columns={'weighted_mortality_rate': 'mortality_rate'},inplace=True)

    for reason in fw_mortality_by_reason:
#         fw_mortality_by_reason[reason].to_csv(f'fw_mortality_{reason}.csv', index=False)
        display(fw_mortality_by_reason[reason].shape)
        display(fw_mortality_by_reason[reason].head())

100%|██████████| 1/1 [01:31<00:00, 91.46s/it]


(672441, 3)

,final_locus_population_id,event_date,mortality_rate
0,36770651,2015-11-21,NaN
1,36770651,2015-11-22,NaN
2,36770651,2015-11-23,NaN
3,36770651,2015-11-24,NaN
4,36770651,2015-11-25,NaN


In [75]:
fw_mortality_by_reason[reason]

,final_locus_population_id,event_date,mortality_rate
0,36770651,2015-11-21,NaN
1,36770651,2015-11-22,NaN
2,36770651,2015-11-23,NaN
3,36770651,2015-11-24,NaN
4,36770651,2015-11-25,NaN
...,...,...,...
672436,194761283,2023-08-12,NaN
672437,194761283,2023-08-13,NaN
672438,194761283,2023-08-14,NaN
672439,194761283,2023-08-15,NaN


In [76]:
if True:
    fw_mortality_final_locus = fw_mortality.merge(
        final_locus_weighted,
        how='inner', 
        left_on=['event_date', 'locus_id'],
        right_on=['event_date', 'historic_locus_id']
    )

    fw_mortality_final_locus['event_year'] = fw_mortality_final_locus['event_date'].dt.year
    fw_mortality_final_locus['weighted_mortality_rate'] = fw_mortality_final_locus['weight'] * fw_mortality_final_locus['mortality_rate']

    fw_mortality_final_locus_grouped = fw_mortality_final_locus.\
    groupby(['final_locus_population_id','event_date'])[['weighted_mortality_rate']]\
    .agg(lambda x: x.sum(skipna=False)).reset_index()

    fw_mortality_final_locus_grouped.rename(columns={'weighted_mortality_rate': 'mortality_rate'},inplace=True)


In [78]:
fw_mortality_final_locus_grouped.head()

,final_locus_population_id,event_date,mortality_rate
0,36770651,2015-09-24,NaN
1,36770651,2015-09-28,NaN
2,36770651,2015-09-30,NaN
3,36770651,2015-10-01,NaN
4,36770651,2015-10-05,NaN


In [79]:
fw_mortality_final_locus_grouped.to_csv('..//data//fw_mortality_final_lp_grouped_no_productiva.csv', index=False)

In [80]:
for reason in tqdm(fw_mortality_by_reason):
    fw_mortality_by_reason[reason] = fw_mortality_by_reason[reason].merge(
        fresh_water_dates,
        left_on='final_locus_population_id',
        right_on='pretransfer_fw_locus_population_id',
        how='inner'
    )

100%|██████████| 1/1 [00:00<00:00,  8.83it/s]


In [81]:
fw_mortality_by_reason[reason].columns

Index(['final_locus_population_id', 'event_date', 'mortality_rate',
       'pretransfer_fw_locus_population_id', 'pretransfer_fw_locus_id',
       'first_movement_date', 'first_feeding_date', 'shipout_date'],
      dtype='object')

In [60]:
for reason in fw_mortality_by_reason:
    display(fw_mortality_by_reason[reason].groupby('final_locus_population_id').count().shape)
    
for _id in fw_mortality_by_reason['Embrionaria']['final_locus_population_id'].unique():
    for reason in mortality_reasons[1:]:
        if _id not in fw_mortality_by_reason[reason]['final_locus_population_id'].unique():
            row = fw_mortality_by_reason['Embrionaria'][
                fw_mortality_by_reason['Embrionaria']['final_locus_population_id'] == _id
            ]
            row['mortality_rate'] = 0.0
            fw_mortality_by_reason[reason] = pd.concat([fw_mortality_by_reason[reason], row])

(4512, 7)

KeyError: 'Embrionaria'

In [ ]:
for reason in fw_mortality_by_reason:
    display(fw_mortality_by_reason[reason].groupby('final_locus_population_id').count().shape)

In [83]:
reason_df = fw_mortality_by_reason[reason]
reason_df.columns

Index(['final_locus_population_id', 'event_date', 'mortality_rate',
       'pretransfer_fw_locus_population_id', 'pretransfer_fw_locus_id',
       'first_movement_date', 'first_feeding_date', 'shipout_date'],
      dtype='object')

In [84]:
fw_mortality_by_reason_full_range = {}

for reason in tqdm(fw_mortality_by_reason):
    reason_df = fw_mortality_by_reason[reason]
    by_id = []
    for _id in reason_df['final_locus_population_id'].unique():
        reason_df_by_id = reason_df[reason_df['final_locus_population_id'] == _id]
        full_range = pd.DataFrame(
            pd.date_range(
                start=reason_df_by_id['first_movement_date'].iloc[0],
                end=reason_df_by_id['shipout_date'].iloc[0]),
            columns=['event_date']
        )
        mortality_full_range = full_range.merge(reason_df_by_id, how='left', on='event_date')
        mortality_full_range['mortality_rate'].fillna(0, inplace=True)
        mortality_full_range['final_locus_population_id'].fillna(_id, inplace=True)
        mortality_full_range['final_locus_population_id'] = mortality_full_range['final_locus_population_id']\
        .astype(int)
        
        by_id.append(mortality_full_range)

    fw_mortality_by_reason_full_range[reason] = pd.concat(by_id)

100%|██████████| 1/1 [00:33<00:00, 33.23s/it]


In [85]:
full_cycles = [
    194359032,
    194527129,
    194497496,
# #     193247058,
# #     194516980,
# #     194546436,
# #     38193840,
# #     194352023,
#     194503079,
# #     193343886,
# #     194530157,
# #     194497483,
# #     194503096,
# #     194522881,
# #     193249474,
# #     193247433,
# #     194523185,
# #     194512165
]

### Non-smoothed

In [86]:
def plot_full_cycle(df: pd.DataFrame,
                    fresh_water_dates_df: pd.DataFrame,
                    full_cycle_id: int,
                    factor: str,
                    reason: str,
#                      save_dir: Path,
                     smoothed: bool = False):
    if df[factor].isna().sum() == df[factor].shape[0]:
        return

    fig, axs = plt.subplots(1, 1)
    sns.lineplot(data=df, x='event_date', y=factor)

    min_ph = df[factor].min()
    first_movement_date = df['event_date'].min()
    hatching_date = df.dropna()['event_date'].min()  # not exact definition of hatching date
    shipout_date = df['event_date'].max()
    first_feeding_date = fresh_water_dates_df[fresh_water_dates_df.pretransfer_fw_locus_population_id==full_cycle_id].first_feeding_date.iloc[0]

    plt.xlim(first_movement_date - datetime.timedelta(days=7), shipout_date + datetime.timedelta(days=7))
    axs.xaxis.set_major_formatter(mdates.DateFormatter(TIME_FORMAT))

    if (hatching_date-first_movement_date).days<7:
        dates_dict = {
            'hatching_date': hatching_date,
            'first_feeding_date': first_feeding_date,
            'shipout_date': shipout_date
        }
    else:
        dates_dict = {
            'first_movement_date': first_movement_date,
            'hatching_date': hatching_date,
            'first_feeding_date': first_feeding_date,
            'shipout_date': shipout_date
        }

    for i, (name, date) in enumerate(dates_dict.items()):
        plt.axvline(date, linestyle='--')
        plt.text(date + datetime.timedelta(days=1), min_ph, f'{name} = {date.strftime(TIME_FORMAT)}', rotation=90)
    
    chart_title = 'Mortality rate vs. time for final_locus_population_id=' + str(full_cycle_id)
    plt.show()
    save_path = save_dir / f'mortality_rate_{reason}_vs_time_{full_cycle_id}.png' if not smoothed else save_dir / f'ph_vs_time_{full_cycle_id}_smoothed.png'
    fig.savefig(save_path)
    plt.close()

In [61]:
save_dir = Path('./')

for full_cycle_id in full_cycles:
    for reason in fw_mortality_by_reason_full_range:
        mortality_full_cycle = fw_mortality_by_reason_full_range[reason][
            fw_mortality_by_reason_full_range[reason]['final_locus_population_id'] == full_cycle_id]
        plot_full_cycle(mortality_full_cycle, fresh_water_dates, full_cycle_id, factor='mortality_rate', reason=reason)

NameError: name 'full_cycles' is not defined

In [88]:
for reason in fw_mortality_by_reason_full_range:
    fw_mortality_by_reason_full_range[reason][
        ['event_date', 'final_locus_population_id', 'mortality_rate']
    ].to_csv(f'../data/mrts/fw_mortality_{reason}.csv', index=False)